In [27]:
import pandas as pd
import numpy as np

In [32]:
with open('./data/RA_SU22_screen/RA_data.csv', 'r', encoding='utf-8-sig') as f: # because of the BOM and excel is stupid
# read in CSV files from data folder as a nunpy array
    data_np = np.genfromtxt(f, delimiter=',', dtype='int')
with open('./data/RA_SU22_screen/RA_labels.csv', 'r', encoding='utf-8-sig') as f: # because of the BOM and excel is stupid
# read in labels of the data
    labels_np = np.genfromtxt(f, delimiter=',', dtype='str')

# reshape both into a single column
data_np = data_np.reshape(-1,1)
labels_np = labels_np.reshape(-1,1)


In [48]:
# create a dataframe from the numpy arrays
df = pd.DataFrame(data_np, columns=['data'])
# add the labels to the dataframe
df['labels'] = labels_np
# label the first 12 lines as '86' and the second 24 as '104' and so on in a column called 'peptide'
df['peptide'] = np.repeat(
    np.concatenate((
    np.repeat(86, 12), np.repeat(104, 12)
    )),
    len(df)/24)

df

,data,labels,peptide
0,81717,positive,86
1,125233,positive,86
2,114981,positive,86
3,194017,,86
4,171267,,86
...,...,...,...
4027,16221,,104
4028,57023,,104
4029,68323,,104
4030,18419,,104
